In [2]:
import os
import numpy as np
from osgeo import gdal

In [5]:
def write_grid_geotiff(input_array, filename, geo_info,
                       cmap='viridis', vmin=0, vmax=75):
    """
    Write a 2D array to a GeoTIFF file.
    
    Parameters
    ----------
    input_array : numpy array
        Grid to write to file.
    filename : str
        Filename for the GeoTIFF.
    geo_info : dict
        contains grid_step, grid_dist, radar_lat, radar_lon
        
    Other Parameters
    ----------------
    cmap : str or matplotlib.colors.Colormap object, optional
        Colormap to use for RGB output or SLD file.
    vmin : int or float, optional
        Minimum value to color for RGB output or SLD file.
    vmax : int or float, optional
        Maximum value to color for RGB output or SLD file.
    """
    
    # Determine whether filename template already contains a suffix
    # If not, append an appropriate one.
    if '.' not in filename:
        name = filename
        end = 'tif'
        ofile = name + "." + end
    else:
        ofile = filename
        
    
    dist      = geo_info['grid_dist']
    rangestep = geo_info['grid_step']
    lat       = geo_info['radar_lat'] #lat origin
    lon       = geo_info['radar_lon'] #lon origin (middle of grid)
    
    # Check if masked array; if so, fill missing data
    data = np.ma.filled(input_array, fill_value=0)
    data = data.astype(float)
    data[data == 0] = np.nan

    iproj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",' + \
        'SPHEROID["WGS84",6378137,298.257223563]],' + \
        'PRIMEM["Greenwich",0],' + \
        'UNIT["degree",0.0174532925199433]],' + \
        'PROJECTION["Azimuthal_Equidistant"],' + \
        'PARAMETER["latitude_of_center",' + str(lat) + '],' + \
        'PARAMETER["longitude_of_center",' + str(lon) + '],' + \
        'PARAMETER["false_easting",0],' + \
        'PARAMETER["false_northing",0],' + \
        'UNIT["metre",1,AUTHORITY["EPSG","4326"]]]'
    out_driver = gdal.GetDriverByName("GTiff")

    # Single-channel, floating-point output
    dst_options = ['COMPRESS=LZW', 'ALPHA=YES']
    dst_ds = out_driver.Create(
        ofile, data.shape[1], data.shape[0], 1,
        gdal.GDT_Float32, dst_options)

    # Common Projection and GeoTransform
    dst_ds.SetGeoTransform([-dist, rangestep, 0, dist, 0, -rangestep])
    dst_ds.SetProjection(iproj)

    # Final output
    dst_ds.GetRasterBand(1).WriteArray(data[::-1, :])
    dst_ds.FlushCache()
    dst_ds = None
    
    os.system('gdalwarp -q -t_srs \'+proj=longlat +ellps=WGS84 ' +
                      '+datum=WGS84 +no_defs\' -dstalpha ' + ofile + ' ' +
                      ofile + '_tmp.tif')
    
    os.system('mv ' + ofile + '_tmp.tif ' + ofile)

In [6]:
#init files
source_ffn = '/home/meso/MEGAsync/projects/PERILS/PERILS_study_03_20181220/radar_data/03_mesh_grid.npy'
dest_ffn   = '/home/meso/MEGAsync/projects/PERILS/hailtrack/IDR03_20181220_mesh.tif'
#init geo info
geo_info = {}
#IDR66 -27.7178	153.24
#IDR03 -34.2624	150.8751
geo_info['radar_lat'] = -34.2624
geo_info['radar_lon'] =	150.8751
geo_info['grid_dist'] = 150000 #maximum grid range (km)
geo_info['grid_step'] = 1000 #grid step (km)

#load grid
input_array = np.load(source_ffn)
rain = False
zr_a = 79 #81 IDR03 79 IDR66
zr_b = 1.6 #1.8 IDR03 1.6 IDR66
if rain:
    print('converting relf to rainrate')
    #limit to max_refl
    refl_array = input_array.copy()
    #convert to z
    refl_array_z = 10. ** (np.asarray(refl_array)/10.)
    rain_array  = (refl_array_z / zr_a) ** (1. / zr_b)
    write_array = rain_array
else:
    write_array = input_array
#write to grid
write_grid_geotiff(write_array, dest_ffn, geo_info,
                       cmap='viridis', vmin=0, vmax=150)
print('complete')

complete
